In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import textstat
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
%matplotlib inline

In [2]:
conn = sqlite3.connect(os.path.join('..', 'raw_data','pitchfork.sqlite'))
df_reviews = pd.read_sql('Select * from reviews',conn)
df_content = pd.read_sql('Select * from content',conn)
df_genres = pd.read_sql('Select * from genres', conn)

### Clean up reviews data

In [3]:
df_reviews.reviewid.value_counts()

9460     2
9417     2
9499     2
9505     2
2047     1
        ..
21792    1
11551    1
9502     1
15645    1
2049     1
Name: reviewid, Length: 18389, dtype: int64

In [4]:
df_reviews = df_reviews.drop_duplicates(subset=['reviewid']).reset_index(drop=True)
df_reviews.head(2)

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017


### Clean up metacritic data

In [6]:
df_m = pd.read_csv(os.path.join('..', 'raw_data','metacritic_complete.csv'))
df_m.head(2)

,artist,album,year,critic_score,user_score,comment
0,melochrome,NaN,NaN,NaN,NaN,artist not found!
1,jets to brazil,NaN,NaN,NaN,NaN,artist not found!


In [7]:
# drop nans and tbds
idx_not_null = (~df_m['critic_score'].isna() & ~df_m['user_score'].isna())
idx_not_tbd = (df_m['critic_score']!='tbd') & (df_m['user_score']!='tbd')
df_m = df_m.loc[idx_not_null & idx_not_tbd]

# convert to float
df_m['critic_score'] = df_m['critic_score'].astype(np.float64)
df_m['user_score'] = df_m['user_score'].astype(np.float64)

In [8]:
df_m.head(2)

,artist,album,year,critic_score,user_score,comment
14,the the,soul mining (30th anniversary deluxe edition) ...,"Jul 1, 2014",92.0,8.1,NaN
15,the the,nakedself,"Feb 29, 2000",75.0,7.4,NaN


### Keep Pitchfork reviews that are only tied to one genre (so that we can easily use genre as a catgeorical var)

In [9]:
df_genres['count_of_genre'] = df_genres.groupby(['reviewid'])['genre'].transform('nunique')
df_genres_1 = df_genres.loc[df_genres['count_of_genre']==1].copy()
df_genres_1.shape

(12148, 3)

In [10]:
df_genres_1.reviewid.value_counts()

9417     2
18431    1
21808    1
1338     1
7481     1
        ..
12955    1
15002    1
8857     1
21143    1
16384    1
Name: reviewid, Length: 12147, dtype: int64

In [11]:
df_genres_1 = df_genres_1.drop_duplicates(subset=['reviewid']).reset_index(drop=True)
df_genres_1.head(2)

,reviewid,genre,count_of_genre
0,22703,electronic,1
1,22721,metal,1


### Join pitchfork reviews with genre, and then with metacritic dataframe

In [12]:
df_p = pd.merge(df_reviews,
                df_genres_1[['reviewid','genre']],
                on='reviewid')
df_p.shape

(12147, 14)

In [13]:
assert(df_p.reviewid.value_counts()==1).all()

In [14]:
df_joined = pd.merge(df_p,
                     df_m,
                     left_on=['artist','title'],
                     right_on=['artist','album'])

df_joined = df_joined.rename(columns={'score':'p_score',
                                      'critic_score':'m_critic_score',
                                      'user_score':'m_user_score'})

df_joined = df_joined.drop(['title'],axis=1)

df_joined.head(2)

,reviewid,artist,url,p_score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,album,year,m_critic_score,m_user_score,comment
0,22704,little simz,http://pitchfork.com/reviews/albums/22704-litt...,7.1,0,katherine st. asaph,contributor,2017-01-05,3,5,1,2017,rap,stillness in wonderland,"Dec 16, 2016",84.0,8.3,NaN
1,22714,brian eno,http://pitchfork.com/reviews/albums/22714-refl...,7.7,0,andy beta,contributor,2017-01-04,2,4,1,2017,experimental,reflection,"Jan 1, 2017",75.0,7.6,NaN


### Add readability score to review content

In [18]:
%%time
df_content['reading_ease'] = df_content['content'].apply(lambda row: textstat.flesch_reading_ease(row))

CPU times: user 1min 6s, sys: 391 ms, total: 1min 7s
Wall time: 1min 8s


### Export

In [21]:
df_m.to_csv(os.path.join('..','data','df_m.csv'),index=False)
df_joined.to_csv(os.path.join('..','data','df_joined.csv'),index=False)
df_reviews.to_csv(os.path.join('..','data','df_reviews.csv'),index=False)
df_content.to_csv(os.path.join('..','data','df_content.csv'),index=False)
df_genres.to_csv(os.path.join('..','data','df_genres.csv'),index=False)